In [ ]:
import os
import cv2
import glob
import json
import tqdm
import numpy as np
from scipy.spatial.transform import Slerp, Rotation
import trimesh
import torch

In [ ]:
def generate_poses(thetas, phis, device, radius=1):
    """ generate poses from an orbit camera
    Args:
        size: batch size of generated poses.
        device: where to allocate the output.
        radius: camera radius
        thetas: [min, max], should be in [0, \pi]
        phis: [min, max], should be in [0, 2\pi]
    Return:
        poses: [size, 4, 4]
    """
    size = thetas.shape[0]

    def normalize(vectors):
        return vectors / (torch.norm(vectors, dim=-1, keepdim=True) + 1e-10)

    centers = torch.stack([
        radius * torch.sin(thetas) * torch.sin(phis),
        radius * torch.cos(thetas),
        radius * torch.sin(thetas) * torch.cos(phis),
    ], dim=-1) # [B, 3]

    # lookat
    forward_vector = - normalize(centers)
    up_vector = torch.DoubleTensor([0, -1, 0]).to(device).unsqueeze(0).repeat(size, 1) # confused at the coordinate system...
    right_vector = normalize(torch.cross(forward_vector, up_vector, dim=-1))
    up_vector = normalize(torch.cross(right_vector, forward_vector, dim=-1))

    poses = torch.eye(4, dtype=torch.float, device=device).unsqueeze(0).repeat(size, 1, 1)
    poses[:, :3, :3] = torch.stack((right_vector, up_vector, forward_vector), dim=-1)
    poses[:, :3, 3] = centers

    return poses


In [ ]:
t_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
num_frames = 150
t_phis = torch.tensor(np.array(np.arange(2*np.pi, 0, -(np.pi/num_frames)*2)), device=t_device, dtype=torch.double)
t_theta = torch.tensor(np.pi*(1.0/3.0), dtype=torch.double)
t_thetas = torch.ones_like(t_phis, dtype=torch.double, device=t_device) * t_theta
# t_radius = torch.ones_like(t_phis, dtype=torch.double, device=t_device) * 3.5 * (0.45/0.55)
t_radius = torch.ones_like(t_phis, dtype=torch.double, device=t_device) * 3.5

decay_first = 1.0
decay_end = 0.6
decay_step = (decay_first - decay_end)/num_frames
decay = torch.tensor(np.array(np.arange(decay_first, decay_end, -decay_step)), device=t_device, dtype=torch.double)
t_radius *= decay

In [ ]:
poses = generate_poses(t_thetas, t_phis, t_device, t_radius)

In [ ]:
exp_name = 'HZ0307_55online'
test_original_path = os.path.join('./data', exp_name, 'transforms_test_original.json')
json_file = 'transforms_test_original.json' if os.path.isfile(test_original_path) else 'transforms_test.json'
data_path = os.path.join('./data', exp_name, json_file)
fp = open(data_path, 'r')
data = json.load(fp)
fp.close()
frame = data['frames'][0]
new_frames = []

In [ ]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [ ]:
def ngp_matrix_to_nerf(pose):
    # for the fox dataset, 0.33 scales camera radius to ~ 2
    new_pose = np.array([
        [pose[2, 0], -pose[2, 1], -pose[2, 2], pose[2, 3]],
        [pose[0, 0], -pose[0, 1], -pose[0, 2], pose[0, 3]],
        [pose[1, 0], -pose[1, 1], -pose[1, 2], pose[1, 3]],
        [0, 0, 0, 1],
    ], dtype=np.float32)
    # new_pose = pose
    return new_pose

In [ ]:
from copy import deepcopy
for pose in poses:
    f = deepcopy(frame)
    f['transform_matrix'] = ngp_matrix_to_nerf(pose.cpu().numpy())
    new_frames.append(f)

In [ ]:
data['frames'] = new_frames

In [ ]:
with open(os.path.join('./data', exp_name, 'transforms_test.json'), 'w') as fp:
    json.dump(data, fp=fp, sort_keys=False, indent=2, cls=NpEncoder)